In [1]:
from IPython.core.display import HTML
from IPython.display import IFrame

#import urllib # package required to interact with live webpage
#import requests

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time
import re

In [2]:
def open_excel(excel_path = 'investment_funds.xlsx'):
    saved_data = pd.read_excel(excel_path)
    return saved_data

In [3]:
site_url = 'https://www.credicorpcapital.com/Colombia/Neg/GA/Paginas/FGA.aspx'    
options = Options()
options.add_argument("--headless") # To Avoid the navigator to open

browser = webdriver.Chrome(options= options)

In [4]:
browser.get(site_url)
time.sleep(2)
default_html = browser.page_source

soup = BeautifulSoup(default_html, 'lxml')

In [11]:
historical_df = open_excel()

temp = pd.DataFrame(columns = historical_df.columns)

In [27]:
temp.columns

Index(['Fondo de Inversion', 'Fecha Extracción', 'Valor de la unidad',
       'Valor en Pesos', '7 días', '30 días', '180 días', 'Año corrido',
       'Último año', 'Últimos dos años', 'Últimos tres años',
       'Fecha de Cierre', 'Fondo administrador por', 'Calificación', 'Plazo'],
      dtype='object')

In [12]:
df = pd.read_html(default_html)

In [40]:
closing_date_df = pd.DataFrame(df[1]).loc[0,:].T
closing_date_df

0    Fecha Cierre
1      2021-07-23
Name: 0, dtype: object

In [42]:
fund_value_df = pd.DataFrame(df[1]).loc[1,:].T
fund_value_df

0    Valor del Fondo
1     74786347512.33
Name: 1, dtype: object

In [94]:
fund_info = pd.concat([pd.DataFrame(df[2]).loc[0:1,:],closing_date_df,fund_value_df], axis = 1)

In [95]:
fund_info.columns = fund_info.loc[0,:]
fund_info.drop(0,axis = 0, inplace = True)
fund_info

,Tipo de participación,Valor TP (en MM),Valor Unidad TP,Último Mes,Últimos 6 Meses,Año Corrido,Último Año,Últimos 2 Años,Últimos 3 Años,Fecha Cierre,Valor del Fondo
1,A,"$68,176.43","$23,276.885027",39.29%,30.82%,33.47%,34.23%,17.92%,13.31%,2021-07-23,74786347512.33


In [96]:
general_info = pd.DataFrame(df[3]).T
general_info.columns = general_info.loc[0,:]
general_info.drop(0, axis = 0, inplace = True)
general_info

,Perfil,Tipo,Objetivo,Horizonte de la Inversión,Politica de Inversión,Inicio de Operaciones,Inversión Mínima,​Valor Mínimo Adiciones y Retiros,​Saldo Mínimo de Permanencia,Retiros,​Cancelaciones,Comisión,Ticker Bloomberg
1,AGRESIVO,Abierto sin pacto de permanencia,Busca el crecimiento del capital por medio de ...,Superior a dos años.,Arquitectura abierta que nos permite seleccion...,08-Junio-10,"$1,000,000","​$500,000​","​$1,000,000",T+5 - Hasta el 95% del saldo incluida la reten...,​T+5,*De acuerdo a la clase de participación,CAMACGA


In [97]:
fund_info['Fondo de Inversion'] = 'Acciones Globales'
fund_info['Calificación'] = 'No Aplica'
fund_info['Plazo'] = 'No aplica'
fund_info['Fondo administrador por'] = 'Credicorp Capital'
fund_info['7 días'] = np.nan

In [98]:
fund_info

,Tipo de participación,Valor TP (en MM),Valor Unidad TP,Último Mes,Últimos 6 Meses,Año Corrido,Último Año,Últimos 2 Años,Últimos 3 Años,Fecha Cierre,Valor del Fondo,Fondo de Inversion,Calificación,Plazo,Fondo administrador por,7 días
1,A,"$68,176.43","$23,276.885027",39.29%,30.82%,33.47%,34.23%,17.92%,13.31%,2021-07-23,74786347512.33,Acciones Globales,No Aplica,No aplica,Credicorp Capital,NaN


In [99]:
fund_info.drop(['Tipo de participación','Valor TP (en MM)'], axis = 1, inplace = True)

In [100]:
fund_info

,Valor Unidad TP,Último Mes,Últimos 6 Meses,Año Corrido,Último Año,Últimos 2 Años,Últimos 3 Años,Fecha Cierre,Valor del Fondo,Fondo de Inversion,Calificación,Plazo,Fondo administrador por,7 días
1,"$23,276.885027",39.29%,30.82%,33.47%,34.23%,17.92%,13.31%,2021-07-23,74786347512.33,Acciones Globales,No Aplica,No aplica,Credicorp Capital,NaN


In [101]:
fund_info.rename(columns = {'Valor Unidad TP':'Valor de la unidad',
                            'Último Mes':'30 días',
                            'Últimos 6 Meses':'180 días',
                           'Año Corrido':'Año corrido',
                            'Último Año':'Último año',
                            'Últimos 2 Años':'Últimos dos años',
                           'Últimos 3 Años':'Últimos tres años',
                            'Fecha Cierre':'Fecha de Cierre',
                            'Valor del Fondo':'Valor en Pesos'}
                 , inplace = True)

In [102]:
fund_info

,Valor de la unidad,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Valor en Pesos,Fondo de Inversion,Calificación,Plazo,Fondo administrador por,7 días
1,"$23,276.885027",39.29%,30.82%,33.47%,34.23%,17.92%,13.31%,2021-07-23,74786347512.33,Acciones Globales,No Aplica,No aplica,Credicorp Capital,NaN


In [111]:
fund_info = fund_info.apply(lambda x: x.str.replace('$','') if x.dtype == 'object' else x, axis = 0)
fund_info = fund_info.apply(lambda x: x.str.replace('%','') if x.dtype == 'object' else x, axis = 0)
fund_info = fund_info.apply(lambda x: x.str.replace(',','') if x.dtype == 'object' else x, axis = 0)

In [112]:
fund_info

,Valor de la unidad,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Valor en Pesos,Fondo de Inversion,Calificación,Plazo,Fondo administrador por,7 días
1,23276.885027,39.29,30.82,33.47,34.23,17.92,13.31,2021-07-23,74786347512.33,Acciones Globales,No Aplica,No aplica,Credicorp Capital,NaN


In [115]:
float_columns = ['Valor de la unidad', '30 días', '180 días', 'Año corrido', 'Último año', 'Últimos dos años',
                 'Últimos tres años','Valor en Pesos']

In [119]:
fund_info[float_columns] = fund_info[float_columns].apply(lambda x: x.astype('float'), axis = 0)

In [122]:
fund_info['Fecha de Cierre'] = pd.to_datetime(fund_info['Fecha de Cierre'], format = '%Y/%m/%d')
fund_info['Fecha Extracción']= pd.to_datetime(time.strftime('%Y/%m/%d', time.localtime(time.time())))

In [124]:
fund_info

,Valor de la unidad,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Valor en Pesos,Fondo de Inversion,Calificación,Plazo,Fondo administrador por,7 días,Fecha Extracción
1,23276.885027,39.29,30.82,33.47,34.23,17.92,13.31,2021-07-23,7.478635e+10,Acciones Globales,No Aplica,No aplica,Credicorp Capital,NaN,2021-07-24


In [125]:
 # Re ordering the columns
fund_info = fund_info[['Fondo de Inversion','Fecha Extracción','Valor de la unidad','Valor en Pesos','7 días',
                       '30 días','180 días','Año corrido','Último año','Últimos dos años','Últimos tres años',
                       'Fecha de Cierre','Fondo administrador por','Calificación','Plazo',]]

In [126]:
fund_info

,Fondo de Inversion,Fecha Extracción,Valor de la unidad,Valor en Pesos,7 días,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Fondo administrador por,Calificación,Plazo
1,Acciones Globales,2021-07-24,23276.885027,7.478635e+10,NaN,39.29,30.82,33.47,34.23,17.92,13.31,2021-07-23,Credicorp Capital,No Aplica,No aplica


In [127]:
browser.quit()

In [128]:
historical_df = pd.concat([historical_df, fund_info]).reset_index(drop = True)

In [131]:
print('upgrading file...')
historical_df.to_excel('investment_funds.xlsx',index = False)
print('file successfully upgraded')

upgrading file...
file successfully upgraded
